# 实验内容
https://www.youtube.com/watch?v=CbTU92pbDKw&list=PLzYUjRAdu5V_J0wClDakA95Qwi2uSj5oX&index=3

### 库

In [106]:
import os
import pandas as pd
from rich.console import Console
from rich.theme import Theme
from rich.table import Table
console = Console()

@2022-12-22 中午
- diff值的日志，需要梳理一下，分不同的段落，整合成一个list，然后依次用于训练数据集
- 新的数据以[0 0 0 0 0 0 ... 1]为界，分成不同的段落

In [107]:
DIR = os.path.abspath('')
# 读取diff日志文件
read_diff = open(DIR+'\\diff-2022-12-21.csv', 'r', encoding='utf8')
diff = []
blank_line = 0
for line in read_diff:
    line = line.strip()
    if len(line) > 0:
        _line = line.split(',')
        diff.append(_line)
    else:
        blank_line += 1        
read_diff.close()

diff2 = []
for each in diff:
    if len(each) > 21:
        each = each[-21:]
    diff2.append(each)

# print(f'空行数量：{blank_line}')
# print(len(diff2))

_remove = []
_border = ['0'] * 20
diff3 = []
_diff3 = []
j = 20
for each in diff2:
    print(each)
    if each[:-1] == _border and j == 0:
        j = 20
        print('封箱')
        diff3.append(_diff3)
        _diff3 = []
    if j == 0:
        print('添加')
        _diff3.append(each)
    else:
        if each[:-1] == _border:
            j = 19
            print('重置计数')
        else:
            print('跳过')
            j -= 1
    

diff3.append(_diff3)

for each in diff3:
    print(len(each))

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
重置计数
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2']
跳过
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0']
跳过
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0', '1']
跳过
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0', '1', '0']
跳过
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0', '1', '0', '-1']
跳过
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0', '1', '0', '-1', '2']
跳过
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0', '1', '0', '-1', '2', '-2']
跳过
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '0', '1', '0', '-1', '2', '-2', '-2']
跳过
['0', '0',

In [108]:
header = ['time']
for i in range(20):
    header.append('col_'+str(i+1))
header.append('result')

diff4 = []
for each in diff3:
    date_ = []
    for i in range(len(each)):
        date_.append(i+1)

    _diff4 = []
    for i in range(len(each)):
        # console.print(date_[i])
        # console.print(diff[i][-21:])
        temp_ = [date_[i]] + each[i]
        _diff4.append(temp_)

    diff4.append(_diff4)
    


In [109]:
import numpy as np
def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  dates = df_as_np[:, 0]
  middle_matrix = df_as_np[:, 1:-1]
  X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

  Y = df_as_np[:, -1]

  return dates, X.astype(np.float32), Y.astype(np.float32)


import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
import keras.metrics as keras_metrics

model = Sequential([layers.Input((20, 1)),
                    layers.LSTM(128),
                    layers.Dense(64, activation='relu'),
                    layers.Dense(64, activation='relu'),
                    layers.Dense(1)])

model.compile(loss='mse', 
              optimizer=Adam(learning_rate=0.0005),
              metrics=['mean_absolute_error'])

mae_log = open(DIR+'\\model\\RNN-4.log', 'w', encoding='utf8')
for k in range(len(diff4)):
  if len(diff4[k]) > 0:
    df = pd.DataFrame(diff4[k], columns=header)
    # print(df)
    dates, X, y = windowed_df_to_date_X_y(df)
    # print(f'Shape = {dates.shape},{X.shape}, {y.shape}')
    q_80 = int(len(dates) * .8)
    q_90 = int(len(dates) * .9)
    dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]
    dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
    dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]
    # plt.plot(dates_train, y_train)
    # plt.plot(dates_val, y_val)
    # plt.plot(dates_test, y_test)
    # plt.legend(['Train', 'Validation', 'Test'])

    if k >= 1:
      test_predictions = model.predict(X).flatten()
      print(type(test_predictions))
      v = test_predictions.tolist()
      print(v[-1])
      mae_ = float(keras_metrics.mean_absolute_error(y, test_predictions))
      mae_log.write(f'第{k-1}次训练后，新的数据集测试\n')
      mae_log.write(f'误差范围：{mae_}\n')
      y_ = y.tolist()
      test_predictions_ = test_predictions.tolist()
      for i in range(len(y_)):
          if abs(y_[i] - (test_predictions_[i]+mae_)) <= 1:
              mae_log.write(f'胜 | 实际值：{y_[i]} | 预测值（含误差范围）：{test_predictions_[i]+mae_}\n')
          else:
              mae_log.write(f'负 | 实际值：{y_[i]} | 预测值（含误差范围）：{test_predictions_[i]+mae_}\n')
          
    # print(f'开始前的权重：{model.get_weights()}')
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)
    # print(f'训练后的权重：{model.get_weights()}')

mae_log.close()

Epoch 1/100
5/5 [==============================] - 1s 74ms/step - loss: 3.6062 - mean_absolute_error: 1.5650 - val_loss: 4.1595 - val_mean_absolute_error: 1.6627
Epoch 2/100
5/5 [==============================] - 0s 14ms/step - loss: 3.5384 - mean_absolute_error: 1.5551 - val_loss: 4.0809 - val_mean_absolute_error: 1.6570
Epoch 3/100
5/5 [==============================] - 0s 14ms/step - loss: 3.4768 - mean_absolute_error: 1.5465 - val_loss: 3.9948 - val_mean_absolute_error: 1.6500
Epoch 4/100
5/5 [==============================] - 0s 15ms/step - loss: 3.3958 - mean_absolute_error: 1.5318 - val_loss: 3.8834 - val_mean_absolute_error: 1.6406
Epoch 5/100
5/5 [==============================] - 0s 23ms/step - loss: 3.2931 - mean_absolute_error: 1.5163 - val_loss: 3.7261 - val_mean_absolute_error: 1.6279
Epoch 6/100
5/5 [==============================] - 0s 14ms/step - loss: 3.1534 - mean_absolute_error: 1.4946 - val_loss: 3.5121 - val_mean_absolute_error: 1.6106
Epoch 7/100
5/5 [===========

In [110]:
import time

current_time = time.strftime("%Y-%m-%d", time.localtime(time.time()))
current_time

model.save(DIR+"\\model\\"+current_time+'-RNN-4.h5')

In [ ]:
train_predictions = model.predict(X_train).flatten()

plt.plot(dates_train[-20:], train_predictions[-20:])
plt.plot(dates_train[-20:], y_train[-20:])
plt.legend(['Training Predictions', 'Training Observations'])

In [ ]:
val_predictions = model.predict(X_val).flatten()

plt.plot(dates_val[-20:], val_predictions[-20:])
plt.plot(dates_val[-20:], y_val[-20:])
plt.legend(['Validation Predictions', 'Validation Observations'])

In [ ]:
test_predictions = model.predict(X_test).flatten()
import keras.metrics as keras_metrics
mae_ = float(keras_metrics.mean_absolute_error(y_test, test_predictions))
print(f'误差范围：{mae_}')
y_ = y_test.tolist()
test_predictions_ = test_predictions.tolist()
for i in range(len(y_)):
    print()
    if abs(y_[i] - (test_predictions_[i]+mae_)) <= 1:
        print(f'胜 | 实际值：{y_[i]} | 预测值（含误差范围）：{test_predictions_[i]+mae_}')
    else:
        print(f'负 | 实际值：{y_[i]} | 预测值（含误差范围）：{test_predictions_[i]+mae_}')
    
plt.plot(dates_test[-20:], test_predictions[-20:])
plt.plot(dates_test[-20:], y_test[-20:])
plt.legend(['Testing Predictions', 'Testing Observations'])

In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Training Predictions', 
            'Training Observations',
            'Validation Predictions', 
            'Validation Observations',
            'Testing Predictions', 
            'Testing Observations'])